In [ ]:
import os
import random  # Holds up spork
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

CROPPED_DIR = "data/CroppedYale"
UNCROPPED_DIR = "data/yalefaces"

In [ ]:
def get_all_files(parent_dir):
    x = [i for i in os.walk(parent_dir)]
    print(len(x))
    subdirs = [i for i in x[1:]]
    paths = []
    for subdir in subdirs:
        paths += [os.path.join(subdir[0], i) for i in subdir[2]]
        
    return paths
        
cropped = get_all_files(CROPPED_DIR)
original = len(cropped)
cropped = [i for i in cropped if "bad" not in i]
print("{} \"*.bad\" files removed.".format(original - len(cropped)))

In [ ]:
def read_pgm(pgm):
    with Image.open(pgm) as f:
        return np.asarray(f)

def read_flat_pgm(pgm):
    return np.ravel(np.asarray(Image.open(pgm)))

In [ ]:
read_pgm(cropped[0])
image_shape = read_pgm(cropped[0]).shape
print("Image Shape: {}".format(image_shape))

In [ ]:
x = read_flat_pgm(cropped[0])

In [ ]:
x.shape

In [ ]:
y = read_flat_pgm(cropped[1])

In [ ]:
np.vstack((x,y)).shape

In [ ]:
data_matrix = np.empty((x.shape[0], len(cropped)))
for col, pgm in enumerate(cropped):
    data_matrix[:, col] = read_flat_pgm(pgm)

# Sanity Check
i = random.randint(0, len(cropped))
assert(data_matrix[:, i].all() == read_flat_pgm(cropped[i]).all())

In [ ]:
u, s, vh = np.linalg.svd(data_matrix, full_matrices=False)

In [ ]:
plt.figure()
plt.stem(s, use_line_collection=True)
plt.title("Modal Energy")
plt.xlabel("Mode Number")
plt.ylabel("Mode Energy")

plt.figure()
plt.stem(np.log(s), use_line_collection=True)
plt.title("Log of Modal Energy")
plt.xlabel("Mode Number")
plt.ylabel("Mode Energy")

In [ ]:
def reconstruction(u, s, vh, modes=None):
    if modes is None:
        modes = s.shape[0]
        
    print("u.shape: {}; vh.shape: {}".format(u.shape, vh.shape))
    s_diag = np.zeros((u.shape[0], vh.shape[0]))
    s_diag[:u.shape[1], :u.shape[1]] = np.diag(s)
    
    return np.matmul(np.matmul(u[:,0:modes], s_diag[0:modes, 0:modes]), vh[0:modes, :])

def svd_and_reconstruction(data_matrix, modes=None):
    u, s, vh = np.linalg.svd(data_matrix, full_matrices=False)
    return reconstruction(u, s, vh, modes=modes)

In [ ]:
def show_reduced_image_plt(reduced_dataset, index):
    img = np.reshape(reduced_dataset[:, index], image_shape)
    plt.figure()
    plt.imshow(img, cmap="gray")
    plt.show()
    
def show_reduced_image_pil(reduced_dataset, index):
    img = np.reshape(reduced_dataset[:, index], image_shape)
    tmp = Image.fromarray(img)
    tmp.show()

In [ ]:
recon = reconstruction(u, s, vh, modes=1)
show_reduced_image_plt(recon, 127)

In [ ]:
def mode_subplots(u, s, vh, image_index, mode_list):
    mode_list = list(set(mode_list))  # remove dupes
    mode_list = [i for i in mode_list if i > 0]
    mode_list.sort()
    num_images = len(mode_list)
    SUBPLOT_COLS = 2
    subplot_rows = math.ceil(num_images)
    
    fig = plt.figure(figsize=(10,10))
    fig.subplots_adjust(wspace=0.01, hspace=0.1)
    #fig.tight_layout()
    for i in range(num_images):
        ax = fig.add_subplot(subplot_rows, SUBPLOT_COLS, i+1)
        ax.title.set_text("Modes: {}".format(mode_list[i]))
        recon = reconstruction(u, s, vh, mode_list[i])
        img = np.reshape(recon[:, image_index], image_shape)
        ax.imshow(img, cmap="gray")

In [ ]:
modes_to_plot = [1, 10, 100, 1000]
images_to_plot = [0, 100, 127, 255, 583]
images_to_plot = [127]

for i in images_to_plot:
    mode_subplots(u, s, vh, i, modes_to_plot)
